## **Samples: Multi-AI Agents wey dey help book hotel**

For dis fast-paced world wey we dey, to plan business trip no be just to book flight and hotel room. E need better coordination and efficiency wey fit hard to achieve. Na here Multi-AI Agents go help, dem dey change how we dey manage our travel needs.

Imagine say you get team of smart agents wey dey work together to handle every part of your trip wella and without wahala. With our advanced AI technology, we don create special agents wey dey help book things and arrange itinerary, so your travel go smooth and stress-free.

Dis na simple example. When we dey plan business trip, we go need talk to business travel agent to get air ticket info, hotel info, and other things. With AI Agents, we fit create agents wey go help book things and agents wey go arrange itinerary, make dem work together to make everything smarter.


# Start Azure AI Agent Service and collect config info from **.env**

### **.env**

Make .env file

**.env** get di connection string for Azure AI Agent Service, di model wey AOAI dey use, and di Google API Search service API, ENDPOINT, etc.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Di name for your Azure AI Agent Service Model Deployment"

[**NOTE**] You go need model wey get 100,000 Rate Limit (Tokens per minute) and Rate Limit of 600 (Request per minute)

  You fit find model for Azure AI Foundry - Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Di connection string for your Azure AI Agent Service Project"

  You fit find di project connection string for di project overview for AI Foundry Portal Screen.

- **SERPAPI_SEARCH_API_KEY** = "Di SERPAPI Search API KEY wey you get"
- **SERPAPI_SEARCH_ENDPOINT** = "Di SERPAPI Search Endpoint wey you get"

To find di Model Deployment Name and Project Connection String for Azure AI Agent Service, you go need create Azure AI Agent Service. E better make you use [dis template](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) to create am directly （***Note:*** Azure AI Agent Service dey available for limited region for now. E better make you check [dis link](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) to set di region)

Agent go need access SERPAPI. E better make you register using [dis link](https://serpapi.com/searches). After you register, you go fit get unique API KEY and ENDPOINT.


# Login to Azure

You go need to login for Azure. Open terminal for VScode and run `az login` command.


# Setup

To run dis notebook, you go need to install di following libraries. Here be di list of di libraries wey you go need and di pip install commands wey follow:

azure-identity: For Azure authentication.  
requests: For to make HTTP requests.  
semantic-kernel: For di semantic kernel framework (if e be custom or specific library, you fit need to install am from one specific source or repository).  


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Explanation: 
import asyncio: Dis one dey bring asyncio module wey go help you do work wey no dey block for Python. E go allow you write code wey fit run at di same time using async and await syntax.  
from typing import Annotated: Dis one dey bring Annotated type from typing module. Annotated na for add extra info to type hints, e fit help for things like validation, documentation, or tools.


In [ ]:
import asyncio,os
from typing import Annotated

# Explanation:
If you use `from dotenv import load_dotenv` and `load_dotenv()`, e go easy to manage configuration settings and sensitive information (like API keys and database URLs) for inside `.env` file. Dis one go help you keep dem separate from your source code, make your application more secure, and e go dey easy to configure.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Explanation:

Import Statement: from azure.identity.aio import DefaultAzureCredential: Dis one dey import DefaultAzureCredential class from azure.identity.aio module. Di "aio" wey dey di module name mean say e dey for asynchronous operations.

Purpose of DefaultAzureCredential: DefaultAzureCredential class na part of di Azure SDK for Python. E dey provide default way to take authenticate wit Azure services. E go try authenticate wit different methods one by one, like environment variables, managed identity, and Azure CLI credentials.

Asynchronous Operations: Di "aio" module mean say DefaultAzureCredential class fit support asynchronous operations. Dis one mean say you fit use am wit asyncio to do authentication requests wey no go block di program.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Explanation:
E dey import different modules and classes from di semantic_kernel package. Make we break am down:

AgentGroupChat from semantic_kernel.agents: Dis class dey handle group chat functionalities for AI agents. AzureAIAgent and AzureAIAgentSettings from semantic_kernel.agents.azure_ai

AzureAIAgent: Dis class na for create and manage AI agents wey dey use Azure AI services.

AzureAIAgentSettings: Dis class na for configure di settings for di AzureAIAgent. TerminationStrategy from semantic_kernel.agents.strategies.termination.termination_strategy:

Dis class dey define strategies for how AI agents go stop execution under some conditions. ChatMessageContent from semantic_kernel.contents.chat_message_content:

Dis class na for handle di content of chat messages.
AuthorRole from semantic_kernel.contents.utils.author_role:

Dis class dey define different roles for authors for di context of chat messages. 

kernel_function from semantic_kernel.functions.kernel_function_decorator: Dis decorator na for define kernel functions, wey be functions wey fit run inside di semantic kernel framework.
All dis imports dey set up di things wey you need to create and manage AI agents wey fit interact for group chat environment, maybe for tasks like hotel booking or similar activities.


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Explanation:
Next we go import CodeInterpreterTool class from azure.ai.projects.models module.

CodeInterpreterTool: Dis class na part of Azure AI SDK and e dey used to interpret and run code inside AI projects. E dey provide things wey go help you run code snippets, check code, or join code execution inside AI workflows.  
Dis import go set up wetin you need to take use CodeInterpreterTool for your project, wey fit help for tasks wey involve interpreting and running code anyhow.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Explanation: 
ApprovalTerminationStrategy class dey provide one special way wey AI agent fit take stop e work. Di agent go stop if di last message wey dey e interaction history get di word "saved". Dis one fit dey useful for situations wey dem dey see di agent work as complete once e get confirmation say something don "saved". Define di interaction method. After di reservation plan don save, e fit stop when e receive di saved signal.


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# Explanation:

Dis line of code dey initialize one AzureAIAgentSettings object wit default or predefined settings by calling di create() method. Dis settings object (ai_agent_settings) fit den dey use to configure and manage one AzureAIAgent instance.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# Explanation:
If you import requests library, e go easy to make HTTP requests and connect wit web services for your Python code.


In [ ]:
import requests

# Explanation:
Dis na one variable wey dey keep API key wey you go use take access SERP (Search Engine Results Page) API service. API key na one special identifier wey dem dey use take confirm say na your account dey make request.

'GOOGLE_SEARCH_API_KEY': Dis one na placeholder string. You go need change ''GOOGLE_SEARCH_API_KEY' to your real SERP API key.

Purpose: Di reason for dis line na to keep di API key inside one variable so e fit dey authenticate di requests wey you dey send go di SERP API service. Di API key na wetin you need to fit access di service and do search.

How to Get a SERP API Key: To get di SERP API key, follow dis general steps for https://serpapi.com (di exact steps fit dey different depending on di SERP API service wey you dey use):

Choose a SERP API Service: Plenty SERP API services dey, like SerpAPI, Google Custom Search JSON API, and others. Choose di one wey go work well for wetin you need.

Sign Up for an Account:

Go di website for di SERP API service wey you choose https://www.serpapi.com and register for account. You fit need provide small information and confirm your email address.

Create an API Key:

After you don register, log in to your account and go di API section or dashboard. Look for di option wey go allow you create or generate new API key.
Copy di API Key:

Once di API key don generate, copy am. Dis key na wetin you go use take confirm your requests to di SERP API service.
Replace di Placeholder:

Change di placeholder for your .env file


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# Explanation:
The BookingPlugin class dey provide methods wey person fit use book hotels and flights wit Serpapi.com Google Search API. E go arrange di correct parameters, send API requests, and process di response to return di booking info wey dey relevant. Di API key (SERPAPI_SEARCH_API_KEY) and endpoint (SERPAPI_SEARCH_ENDPOINT) na wetin dem dey use to authenticate and send requests go di Google Search API.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# Explanation:
Di SavePlugin class dey provide one method wey dem call saving_plan to take save trip plans wit Azure AI services. E dey set up Azure credentials, create AI agent, process user inputs to generate and save di trip plan content, and e dey handle file saving plus cleanup operations. Di method go return "Saved" if e complete successfully.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# Explanation:
Dis code dey setup Azure AI agents wey go help book flight and hotel, plus save trip plans based on wetin user talk. E dey use Azure credentials to create and configure di agents, process wetin user talk through group chat, and make sure say e clean up well after di tasks don finish. Di agents dey use specific plugins (BookingPlugin and SavePlugin) to do di work wey dem suppose do.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)


<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Dis dokyument don use AI transleto service [Co-op Translator](https://github.com/Azure/co-op-translator) do di translation. Even as we dey try make am correct, abeg sabi say machine translation fit get mistake or no dey accurate well. Di original dokyument wey dey for im native language na di main correct source. For important mata, e good make una use professional human translation. We no go fit take blame for any misunderstanding or wrong interpretation wey fit happen because una use dis translation.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
